In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import torch
import math
import pickle


In [32]:
#split the dataset

dataset = pickle.load(open('datasets.pickle','rb'))

batchsize = 256
#train_x,test_x,train_y,test_y = train_test_split(torch.tensor(X),torch.tensor(Y),test_size=0.2,stratify=torch.tensor(Y))
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
#trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainset = dataset[0]
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

#valset = torch.utils.data.TensorDataset(val_x,val_y)
valset = dataset[1]
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

#testset = torch.utils.data.TensorDataset(test_x,test_y)

testset = dataset[2]
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)



print(trainset[0][0][:4],trainset[0][1].shape)

input_dim = 4
output_dim = trainset[0][1].shape[0]

tensor([ 209, 5287,    5,    2]) torch.Size([5911])


In [102]:

#define the model

class NCF(torch.nn.Module):

	def __init__(self, in_channels, hidden_channels, out_channels):
		super().__init__()
		self.emb1 = torch.nn.Embedding(100000,32)
		self.emb2 = torch.nn.Embedding(100000,32)
		self.timeday = torch.nn.Linear(2,32)
		self.class1 = torch.nn.Linear(96, hidden_channels)
		self.class2 = torch.nn.Linear(hidden_channels, 32)
		self.class3 = torch.nn.Linear(32,16)
		self.class4 = torch.nn.Linear(16, out_channels)



	def forward(self,data):


		user = self.emb1(data[:,0])
		loc = self.emb2(data[:, 1])
		#print(user.shape,loc.shape)
		hour = data[:,2]
		day = data[:,3]
		timeday = torch.stack([hour,day],dim=-1)
		#print(timeday.shape)
		timeday = self.timeday(timeday.float())
		x = torch.cat((user,loc,timeday),dim=1)
		#print(x.shape)
		x = self.class1(x).relu()
		x = self.class2(x).relu()
		x = self.class3(x).relu()
		x = self.class4(x)

		return x

In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

lr = 0.001
wd = 5e-4

model = NCF(4,64,output_dim).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay=wd)
criterion = torch.nn.CrossEntropyLoss()


Using device: cuda


In [113]:
import tqdm
numepoch = 100

#training phase

with tqdm.tqdm(total=numepoch*len(trainloader), unit='batches') as pbar:
  losses = []
  for epoch in range(numepoch):

    model.train()
    optimizer.zero_grad()

    print(f"EPOCH = {epoch}")

    for batch in trainloader:

      input = batch[0].to(device)
      label = batch[1].to(device)

      #print(input.shape)
      #print(label.shape)
      #one_hot_encode_labels(label)
      #print(input.shape,input[:,:4].shape)

      out = model(input[:,:4])

      #print(out.shape)
      #out = model(input)
      #out=out.squeeze(dim=1)
      #print(out)


      loss = criterion(out,label)
      losses.append(loss)
      loss.backward()
      optimizer.step()



      pbar.set_postfix({'Loss': loss.item()})
      pbar.update()


      #print(df)


  6%|▌         | 18/300 [00:00<00:03, 92.45batches/s, Loss=8.24]

EPOCH = 0
EPOCH = 1
EPOCH = 2
EPOCH = 3
EPOCH = 4
EPOCH = 5
EPOCH = 6


 13%|█▎        | 39/300 [00:00<00:02, 97.90batches/s, Loss=7.46]

EPOCH = 7
EPOCH = 8
EPOCH = 9
EPOCH = 10
EPOCH = 11
EPOCH = 12
EPOCH = 13


 20%|██        | 60/300 [00:00<00:02, 95.63batches/s, Loss=6.25]

...

EPOCH = 88
EPOCH = 89
EPOCH = 90
EPOCH = 91
EPOCH = 92


 97%|█████████▋| 292/300 [00:03<00:00, 73.82batches/s, Loss=0.0819]

EPOCH = 93
EPOCH = 94
EPOCH = 95
EPOCH = 96
EPOCH = 97


100%|██████████| 300/300 [00:03<00:00, 83.85batches/s, Loss=0.0736]

EPOCH = 98
EPOCH = 99


In [ ]:
import matplotlib.pyplot as plt
#print CE loss evolution
ind = [i for i in range(len(losses))]
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.plot(ind,losses)
plt.show()

In [115]:
import tqdm

criterion = torch.nn.CrossEntropyLoss()

#testing phase

with tqdm.tqdm(total=len(testloader), unit='batches') as pbar:
  losses = []

  model.eval()

  for batch in testloader:

    input = batch[0].to(device)
    label = batch[1].to(device)

    out = model(input[:,:4])

    out = out.squeeze(dim=1).to(device)

    loss = criterion(out,label)

    losses.append(loss)

    pbar.set_postfix({'Loss': loss.item()})
    pbar.update()

print(sum(losses)/len(losses))


100%|██████████| 1/1 [00:00<00:00, 123.77batches/s, Loss=21.3]

tensor(21.3053, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
import matplotlib.pyplot as plt

#print loss evolution

ind = [i for i in range(len(losses))]
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.plot(ind,losses)
plt.show()